# LINEの会話履歴から学習用のデータセットづくり
5つまでの履歴をさかのぼる

## ドライブにマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/UserBot

## 必要ライブラリのインストール

In [ ]:
!pip install demoji

## データセットの作成

In [ ]:
import csv
import demoji
import re

#自分のLINE名と会話相手のLINE名を入れる
User1="自分のLINE名"
User2="会話相手のLINE名"

User_dict={User1:"User1",User2:"User2"}

demoji.download_codes()
pattern1 = r"https?://[\w/:%#\$&\?\(\)~\.=\+\-]+"
pattern2 = r"\(.+?\)"
pattern = r"[\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF0-9０-９a-zA-Z、。！？!?…]+"

log=[]
texts=[]
rows=[]
input_text=[]
m_log=""
t_log=""
pre_t_log=""
pre_m_log=""
with open('[LINE] ' + User2 + 'とのトーク.txt') as csvfile:
    reader = csv.reader(csvfile, delimiter='\t', quotechar='"')
    for row in reader:
        if len(row) > 0:
            if len(row) == 1:
                if "202" in row[0]:
                    log.append(row)
            elif row[2] != "[スタンプ]" and row[2] != "[写真]" and row[2] != "[動画]" :
                row[2]=demoji.replace(row[2])
                row[2]=row[2].replace("emoji","")
                temp = re.findall(pattern, row[2])
                text=[]
                for now in temp:
                    if len(now) >= 2:
                        if re.search(r"[\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF]", now):
                            text.append(now)
                row[2]="".join(text)
                if len(row) == 1:
                    print(row)
                if len(row[2]) > 0:
                    log.append(row)

log=log[2:]
result=[]
temp_log=[]
for now in log:
    if len(temp_log)>=5:
        temp_log=temp_log[1:]
    if len(now)==1:
        temp_log=[]
    elif now[1] == User2:
        text="<s>"
        for i,t in enumerate(temp_log):
            text=text+"<input"+str(len(temp_log)-i)+">"+User_dict[t[1]]+":"+t[2]
        text=text+"[SEP]"+User_dict[now[1]]+":"+now[2]+"</s>"
        result.append([text])
        temp_log.append(now)
    else:
      temp_log.append(now)

with open("train.txt", "w") as fout:
    writer = csv.writer(fout)
    for now in result:
        writer.writerows([now])

# GPT-2のファインチューニング

## 必要ライブラリのインストール

In [ ]:
!pip install git+https://github.com/huggingface/transformers
# rinna/japanese-gpt2-mediumのtokenizerはsentencepieceなのでsentencepieceもインストールする必要があります。
!pip install sentencepiece
!pip install datasets
!pip install evaluate

## 追加学習の実行

In [ ]:
! python ./run_clm.py --model_name_or_path=rinna/japanese-gpt2-medium --train_file=train.txt --validation_file=train.txt --do_train --do_eval --num_train_epochs=8 --save_steps=10000 --save_total_limit=3 --per_device_train_batch_size=1 --per_device_eval_batch_size=1 --output_dir=./output --use_fast_tokenizer=False

## 作成したモデルで対話文を生成

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt2-medium")
tokenizer.do_lower_case = True
model = AutoModelForCausalLM.from_pretrained('output/')
model.to(device)
model.eval()

In [ ]:
def generate_return(input, num_gen=1):
    input_text = '<s>'+ input + '[SEP]'
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
    out = model.generate(input_ids, do_sample=True, top_p=0.95, top_k=40, 
                         num_return_sequences=num_gen, max_length=256, bad_words_ids=[[1], [5]])
    print('-'*5, '生成された返答', '-'*5)
    for sent in tokenizer.batch_decode(out):
        sent = sent.split('[SEP]</s>')[1]
        sent = sent.replace('</s>', '')
        print(sent)

In [ ]:
generate_return('おはよう')